In [ ]:
!pip install gradio

In [ ]:
!pip install bitsandbytes
!pip install trainsformers
!pip install peft
!pip install accelerate
!pip install datasets

In [1]:
# tokenizer
from transformers import AutoTokenizer
model_id = "beomi/KoAlpaca-Polyglot-5.8B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,)
# set pad token - to avoid error while training
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from peft import prepare_model_for_kbit_training


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained("/content/gdrive/MyDrive/Colab Notebooks/checkpoint-18500", quantization_config=bnb_config, device_map={"":0})

# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
#Setting the Pretraining_tp to 1 ensures we are using the Linear Layers to the max computation possible

model.config.use_cache = True
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
import re
def url_encode(text: str):
    "url 검출"
    
    # URL 추출할 정규표현식 생성
    url_regex = r"(https?:\/\/)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&\/\/=]*)"

    reg = re.compile(url_regex)
    
    res = reg.search(text)
    
    if res == None:
        return text
    
    else:
        indexes = res.span()
        
        url_txt = text[indexes[0]:indexes[1]]
        
        return url_txt

def clean_text(text):
  text_rmv = re.sub('[-=+,#/\?^@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', text)
  return text_rmv

def gen(x):
    gened = model.generate(
        **tokenizer(
            x,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=1024,
        early_stopping=True,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    gened = tokenizer.decode(gened[0])
    split_gened = gened.split('\n')

    output_text = ""
    idx = 0
    for gen_text in split_gened:
        if gen_text.find('요약:')>=0:
            gen_text = clean_text(gen_text)
            output_text +=f"{idx+1}. {gen_text}\n"

    return output_text

def preprocessing_function_test(examples):
    texts = examples['text']

    descriptions = ""

    for text in texts:
        if len(descriptions) != 0:
            descriptions = descriptions + '\n'
        for text_line in text:
            if len(descriptions) == 0:
                descriptions = text_line['sentence']
            else:
                descriptions = descriptions + ' ' + text_line['sentence']



    prompt = f"제목 : {examples['title']}\n본문: {descriptions}\n요약:"
    tokenized = tokenizer(prompt, truncation=True, max_length=1024)


    return tokenized

In [4]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
def art_crawl(url):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수

    Args:
        url: 크롤링할 사이트 url


    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리

    """
    art_dic = {}

    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"

    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")

    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)

    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)

    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)

    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str

    return art_dic

art_crawl("https://n.news.naver.com/article/022/0003945184?cds=news_media_pc&type=editn")

{'title': '‘손흥민 아버지’ 손웅정, 아동학대 혐의 피소…“고소인 주장과 달라” 반박',
 'date': '2024.06.26. 오전 6:45',
 'main': '손 감독 “사랑 전제된 언행…시대 변화 못 읽은 점 반성”손흥민의 아버지 손웅정 감독이 운영하는 유소년 축구 훈련기관 \'SON축구아카데미\'에서 손 감독과 코치진들이 소속 유소년 선수에 대한 욕설과 체벌 등 아동학대 혐의로 피소됐다.  \xa0  손 감독은 "최근 아카데미 훈련 도중 거친 표현과 체력 훈련 중 이뤄진 체벌에 관해 현재 수사가 진행 중"이라며 "마음의 상처를 받은 아이와 그 가족들에게 깊은 사과의 뜻을 전한다"고 밝혔다.  \n\n\n\n   손웅정 SON축구아카데미 감독. 연합뉴스.    26일 연합뉴스에 따르면 손 감독과 A 코치, B 코치 등 3명은 아동복지법상 아동학대 혐의로 송치돼 검찰 조사를 받는 것으로 확인됐다.  \xa0  이 사건은 지난 3월 19일 아동 C군 측이 "오키나와 전지훈련 중이던 지난 3월 9일 A 코치가 C군의 허벅지 부위를 코너킥 봉으로 때려 2주간 치료가 필요한 상처를 입혔다"고 고소하면서 불거졌다.  \xa0  고소인 측이 경찰 조사에서 진술한 바에 따르면 당시 경기에서 진 C군 팀 선수들은 패배했다는 이유로 A 코치로부터 정해진 시간 내에 골대에서 중앙선까지 20초 안에 뛰어오라는 지시를 받았다고 주장했다.  \xa0  그러나 C군을 비롯한 4명이 제시간에 들어오지 못하자 엎드린 자세로 엉덩이를 코너킥 봉으로 맞았다고 진술했다.  \xa0  손 감독으로부터도 오키나와 전지훈련 기간이었던 지난 3월 7∼12일 훈련 중 실수했다는 이유로 욕설을 들은 것을 비롯해 경기는 물론 기본기 훈련을 잘 못한다는 이유로 욕을 들었다는 내용이 진술에 포함됐다.  \xa0  아카데미 소속 선수들이 함께 사는 숙소에서 B 코치에 의해 엉덩이와 종아리를 여러 차례 맞았고, 구레나룻을 잡아당기거나 머리 부위를 맞았다는 주장도 진술서에 담겼다.  \n\n\n\n 

In [ ]:
# gradio library를 가져온다. gr로 줄여서 칭한다.
import gradio as gr

In [ ]:
remove_enter = ['\n','\n\n','\n\n\n','\n\n\n\n','\n\n\n\n\n','\n\n\n\n\n\n', u'\xa0',]
remove_space = ['  ','   ','    ','     ','       ','        ','         ','          ',]
# 챗봇에 채팅이 입력되면 이 함수를 호출합니다.
# message는 유저의 채팅 메시지, history는 채팅 기록, additional_input_info는 additional_inputs안 블록의 정보를 받습니다.
def response(message, history, additional_input_info):

    url = url_encode(message)

    if url==None:
        return "url을 입력해주세요."

    news = art_crawl(url)

    if len(news['main'])==0:
        return "잘 못된 url 입니다."

    for rm_ent in remove_enter:
        description = news['main'].replace(rm_ent, '\n')

    for rm_spc in remove_space:
        description = description.replace(rm_spc, ' ')

    prompt = f"제목 : {news['title']}\n본문: {description}\n요약:"

    summary = gen(f"요약해줘\n{prompt}")

    return summary

gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="url을 입력해주세요", container=False, scale=7),
        title="뉴스기사를 요약해주는 AI 입니다.",
        description="네이버 뉴스 url을 입력하면 요약해주는 챗봇서비스 입니다",
        theme="soft",
        examples=[["https://n.news.naver.com/article/661/0000041753?type=main"], ["https://n.news.naver.com/article/015/0005002147?cds=news_media_pc&type=editn"], ["https://n.news.naver.com/article/024/0000089905?cds=news_media_pc&type=editn"]],
        retry_btn="다시보내기 ↩",
        undo_btn="이전챗 삭제 ↺",
        clear_btn="전챗 삭제 ✄"
).launch()